# 2025 DATA·AI 분석 경진대회 - 논문·데이터 추천 에이전트 평가

이 노트북은 LLM 기반 연구 데이터/논문 추천 시스템의 성능을 평가합니다.

**실행 환경**
- GPU: NVIDIA RTX 3080 이상
- CUDA: 11.8+
- Python: 3.10+

## 1. 환경 설정 및 라이브러리 임포트

In [1]:
import sys
import os
import json
import logging
from datetime import datetime
from pathlib import Path
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

# 프로젝트 루트 경로
project_root = '/home/infidea/backup-data/paper-reco-agent'
sys.path.insert(0, project_root)

print(f"프로젝트 루트: {project_root}")

# 로깅 설정
os.makedirs(os.path.join(project_root, 'logs'), exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler(os.path.join(project_root, 'logs/evaluation.log'))
    ],
    force=True
)

print("✅ 로깅 설정 완료")

# 평가 모듈 임포트
from src.evaluation.metrics import (
    evaluate_recommendations,
    batch_evaluate,
    format_evaluation_report
)
from src.config.settings import settings

print("✅ 평가 모듈 임포트 완료")

프로젝트 루트: /home/infidea/backup-data/paper-reco-agent
✅ 로깅 설정 완료
✅ 평가 모듈 임포트 완료


/home/infidea/backup-data/miniconda3/envs/paper-agent/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. 테스트 데이터셋 로드

In [2]:
# 테스트 데이터셋 로드
testset_path = os.path.join(project_root, 'data', 'test', 'testset_filtered.json')

with open(testset_path, 'r', encoding='utf-8') as f:
    testset = json.load(f)

print(f"✅ 테스트 데이터셋 로드 완료: {testset_path}")
print(f"📊 테스트 카테고리 수: {len(testset.keys())}")

# 모든 테스트 케이스 추출
all_test_cases = []
for category, cases in testset.items():
    for case in cases:
        case['category'] = category
        all_test_cases.append(case)

print(f"📝 총 테스트 케이스: {len(all_test_cases)}개")
print(f"\n첫 번째 테스트 케이스 예시:")
print(json.dumps(all_test_cases[0], indent=2, ensure_ascii=False)[:500] + "...")

✅ 테스트 데이터셋 로드 완료: /home/infidea/backup-data/paper-reco-agent/data/test/testset_filtered.json
📊 테스트 카테고리 수: 2
📝 총 테스트 케이스: 17개

첫 번째 테스트 케이스 예시:
{
  "input_dataset": {
    "svc_id": "c1f03ab868f32371ec97a650bcdf39ac",
    "title": "한반도 기상 관측 데이터 세트",
    "description": "한반도 종관 기상 관측 세스템으로부터의 관측 데이터 세트",
    "keywords": [
      "한반도",
      "기상",
      "센서데이터"
    ]
  },
  "candidate_pool": {
    "papers": [
      {
        "id": "DIKO0014589705",
        "title": "쿠로시오 해류의 변동성과 동아시아 겨울 기후와의 상관성 연구",
        "abstract": "동아시아 겨울 기후를 대표하는 동아시아 겨울 몬순의 변동성은 외부적 다양한 인자들에 의해 영향을 받는다. 북서태평양에서 주요한 흐름인 쿠로시오 해류도 한반도와 주변 바다에 영향을 미치며 동아시아 기후에 영향을 주는...


## 3. 추천 에이전트 초기화

In [4]:
# 추천 에이전트 임포트 및 초기화
from src.agents.recommendation_agent import KoreanResearchRecommendationAgent
from src.config.settings import settings

print("모델 설정:")
print(f"  - 모델명: {settings.MODEL_NAME}")
print(f"  - 임베딩 모델: {settings.EMBEDDING_MODEL}")
print(f"  - 개발 모드: {settings.DEV_MODE}")
print("\n🚀 에이전트 초기화 중... (수 분 소요될 수 있습니다)")

agent = KoreanResearchRecommendationAgent()

print("\n✅ 에이전트 초기화 완료")
print(f"모델 정보: {json.dumps(agent.llm_model.get_model_info(), indent=2, ensure_ascii=False)}")

2025-10-15 17:11:01,869 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cuda:0
2025-10-15 17:11:01,872 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: intfloat/multilingual-e5-large
모델 설정:
  - 모델명: google/gemma-2-9b-it
  - 임베딩 모델: intfloat/multilingual-e5-large
  - 개발 모드: False

🚀 에이전트 초기화 중... (수 분 소요될 수 있습니다)
2025-10-15 17:11:08,763 - src.agents.recommendation_agent - INFO - 🚀 프로덕션 모드로 실행: 실제 LLM 모델 사용
2025-10-15 17:11:08,765 - src.models.llm_model - INFO - 🚀 Gemma 모델 로딩 시작: google/gemma-2-9b-it
2025-10-15 17:11:08,765 - src.models.llm_model - INFO -    - 디바이스: cuda
2025-10-15 17:11:10,231 - src.models.llm_model - INFO -    - FP16 모드


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 4/4 [00:19<00:00,  4.90s/it]


2025-10-15 17:11:42,559 - src.models.llm_model - INFO - ✅ Gemma 모델 로딩 완료

✅ 에이전트 초기화 완료
모델 정보: {
  "model_name": "google/gemma-2-9b-it",
  "model_type": "Gemma",
  "device": "cuda",
  "dtype": "float16",
  "max_tokens": 512,
  "temperature": 0.1,
  "parameters": "9B",
  "context_length": "8K"
}


## 4. 평가 실행

모든 테스트 케이스에 대해 추천을 생성하고 평가합니다.

In [5]:
# TODO: 추천 개수 설정 (선택사항)
# 최대값: agent의 max_paper_candidates, max_dataset_candidates 값
num_paper_recommendations = 5
num_dataset_recommendations = 5

# agent에서 최대 후보 개수 가져오기
MAX_PAPER_CANDIDATES = agent.max_paper_candidates
MAX_DATASET_CANDIDATES = agent.max_dataset_candidates

print(f"최대 추천 가능 개수: 논문 {MAX_PAPER_CANDIDATES}개, 데이터셋 {MAX_DATASET_CANDIDATES}개")

# 추천 개수 검증
if num_paper_recommendations > MAX_PAPER_CANDIDATES:
    print(f"⚠️  경고: 논문 추천 개수({num_paper_recommendations})가 최대값({MAX_PAPER_CANDIDATES})을 초과합니다.")
    print(f"   자동으로 {MAX_PAPER_CANDIDATES}개로 조정됩니다.")
    num_paper_recommendations = MAX_PAPER_CANDIDATES

if num_dataset_recommendations > MAX_DATASET_CANDIDATES:
    print(f"⚠️  경고: 데이터셋 추천 개수({num_dataset_recommendations})가 최대값({MAX_DATASET_CANDIDATES})을 초과합니다.")
    print(f"   자동으로 {MAX_DATASET_CANDIDATES}개로 조정됩니다.")
    num_dataset_recommendations = MAX_DATASET_CANDIDATES

if num_paper_recommendations < 1:
    print(f"⚠️  경고: 논문 추천 개수는 최소 1개 이상이어야 합니다.")
    num_paper_recommendations = 1

if num_dataset_recommendations < 1:
    print(f"⚠️  경고: 데이터셋 추천 개수는 최소 1개 이상이어야 합니다.")
    num_dataset_recommendations = 1

print(f"✅ 추천 설정: 논문 {num_paper_recommendations}개, 데이터셋 {num_dataset_recommendations}개")

최대 추천 가능 개수: 논문 10개, 데이터셋 10개
✅ 추천 설정: 논문 5개, 데이터셋 5개


In [8]:
# 평가 실행
all_eval_results = []
k_values = [3, 5]

print(f"🔍 {len(all_test_cases)}개 테스트 케이스 평가 시작")
print(f"📏 평가 k 값: {k_values}")
print("=" * 80)

for idx, test_case in enumerate(tqdm(all_test_cases, desc="평가 진행")):
    try:
        dataset_id = test_case['input_dataset']['svc_id']
        category = test_case['category']

        print(f"\n[{idx+1}/{len(all_test_cases)}] {test_case['input_dataset']['title']}")
        print(f"   카테고리: {category}")

        # 추천 생성
        recommendation_result = await agent.recommend(
            dataset_id,
            num_paper_recommendations=num_paper_recommendations,
            num_dataset_recommendations=num_dataset_recommendations
        )

        if 'error' in recommendation_result:
            print(f"   ❌ 오류: {recommendation_result['error']}")
            continue

        # 평가 형식으로 변환
        predictions = []
        for paper in recommendation_result.get('paper_recommendations', []):
            predictions.append({
                'type': 'paper',
                'id': paper['id'],
                'rank': paper['rank'],
                'score': paper['score']
            })

        for dataset in recommendation_result.get('dataset_recommendations', []):
            predictions.append({
                'type': 'dataset',
                'id': dataset['id'],
                'rank': dataset['rank'],
                'score': dataset['score']
            })

        # 평가
        ground_truth = test_case['candidate_pool']
        eval_result = evaluate_recommendations(predictions, ground_truth, k_values)

        # 메타데이터 추가
        eval_result['test_case_index'] = idx
        eval_result['dataset_id'] = dataset_id
        eval_result['category'] = category
        eval_result['input_title'] = test_case['input_dataset']['title']
        eval_result['processing_time_ms'] = recommendation_result.get('processing_time_ms', 0)

        all_eval_results.append(eval_result)

        # 결과 요약 출력 (k_values 반영)
        summary_chunks = []
        for k in k_values:
            ndcg = eval_result.get(f"overall_ndcg@{k}", 0)
            mrr = eval_result.get(f"overall_mrr@{k}", 0)
            recall = eval_result.get(f"overall_recall@{k}", 0)
            summary_chunks.append(
                f"nDCG@{k}={ndcg:.4f}, MRR@{k}={mrr:.4f}, Recall@{k}={recall:.4f}"
            )
        print("   ✅ " + " | ".join(summary_chunks))

    except Exception as e:
        print(f"   ❌ 예외 발생: {e}")
        logging.error(f"테스트 케이스 {idx} 평가 실패: {e}", exc_info=True)
        continue

print("\n" + "=" * 80)
print(f"✅ 평가 완료: {len(all_eval_results)}/{len(all_test_cases)}개 케이스 성공")

🔍 17개 테스트 케이스 평가 시작
📏 평가 k 값: [3, 5]


평가 진행:   0%|          | 0/17 [00:00<?, ?it/s]


[1/17] 한반도 기상 관측 데이터 세트
   카테고리: 국내
2025-10-15 17:24:39,270 - src.agents.recommendation_agent - INFO - 추천 프로세스 시작: 데이터셋 ID c1f03ab868f32371ec97a650bcdf39ac
2025-10-15 17:24:39,367 - httpx - INFO - HTTP Request: GET https://dataon.kisti.re.kr/rest/api/search/dataset/c1f03ab868f32371ec97a650bcdf39ac?key=4936BC43D48603524DEDA2E2D56D6B46 "HTTP/1.1 200 200"
2025-10-15 17:24:39,369 - src.clients.dataon_client - INFO - Successfully retrieved metadata for dataset c1f03ab868f32371ec97a650bcdf39ac
2025-10-15 17:24:39,370 - src.clients.dataon_client - INFO - API Response:
{
  "response": {
    "elapsed time": "35 ms",
    "status": "200",
    "message": "OK",
    "total count": "1",
    "type": "json"
  },
  "records": {
    "svc_id": "c1f03ab868f32371ec97a650bcdf39ac",
    "ctlg_type": "02",
    "dataset_type": "01",
    "ctlg_type_pc": "dataset",
    "dataset_type_pc": "국내",
    "dataset_pub_dt_pc": "2025",
    "dataset_access_type_pc": "공개",
    "file_yn_pc": "직접다운로드",
    "dataset_cc_license


Batches: 100%|██████████| 1/1 [00:00<00:00, 113.27it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 67.92it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 85.10it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 48.95it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 80.14it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 57.21it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 132.50it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 134.36it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 111.92it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 118.22it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 135.62it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 81.98it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 129.39it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 65.28it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 130.36it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 102.86it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 127.53it/s]

Batches: 100%|██████████| 1/1 [00:00<

2025-10-15 17:24:41,529 - src.agents.recommendation_agent - INFO - 상위 15개 논문, 12개 데이터셋 순위 결정 완료
2025-10-15 17:24:41,530 - src.agents.recommendation_agent - INFO - LLM paper 추천 생성 시도 1/2
2025-10-15 17:24:41,530 - src.agents.recommendation_agent - INFO - ================================================================================
2025-10-15 17:24:41,531 - src.agents.recommendation_agent - INFO - LLM에게 전송하는 paper 프롬프트:
2025-10-15 17:24:41,532 - src.agents.recommendation_agent - INFO - # Task: Re-rank and Select Top 5 paper Recommendations
You are a research recommendation expert. Re-rank the candidates and select the top 5 most relevant items.
## Source Dataset:
Title: 한반도 기상 관측 데이터 세트
Description: 한반도 종관 기상 관측 세스템으로부터의 관측 데이터 세트
Keywords: 한반도, 기상, 센서데이터

## Top 10 paper Candidates (by E5+BM25 hybrid score):

[1] ID: DIKO0012379411
- Title: 기상 Index들과 한반도 강수량과의 상관관계분석
- Description: 국 문 초 록 최근 기후변화와 관련하여 전 세계적으로 강수의 빈도, 강도, 공간적 범위, 지속기간에 변화가 나타나 가뭄 등 다양한 사회경제학적 문제점이 발생하고 있다. 물과 관련된 이상

...

   ✅ nDCG@3=0.0000, MRR@3=0.0000, Recall@3=0.0000 | nDCG@5=0.0000, MRR@5=0.0000, Recall@5=0.0000

[17/17] Police recorded crime trends in Victoria during the COVID-19 pandemic
   카테고리: 해외
2025-10-15 17:33:33,781 - src.agents.recommendation_agent - INFO - 추천 프로세스 시작: 데이터셋 ID b89d0c5d5891c5c80cd7a565ce8a3dc6
2025-10-15 17:33:33,886 - httpx - INFO - HTTP Request: GET https://dataon.kisti.re.kr/rest/api/search/dataset/b89d0c5d5891c5c80cd7a565ce8a3dc6?key=4936BC43D48603524DEDA2E2D56D6B46 "HTTP/1.1 200 200"
2025-10-15 17:33:33,888 - src.clients.dataon_client - INFO - Successfully retrieved metadata for dataset b89d0c5d5891c5c80cd7a565ce8a3dc6
2025-10-15 17:33:33,889 - src.clients.dataon_client - INFO - API Response:
{
  "response": {
    "elapsed time": "34 ms",
    "status": "200",
    "message": "OK",
    "total count": "1",
    "type": "json"
  },
  "records": {
    "svc_id": "b89d0c5d5891c5c80cd7a565ce8a3dc6",
    "ctlg_type": "02",
    "dataset_type": "02",
    "ctlg_type_pc": "dataset"


Batches: 100%|██████████| 1/1 [00:00<00:00, 62.69it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 88.15it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 81.21it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 78.45it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 82.17it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 81.45it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 80.60it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 81.22it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 80.11it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 78.43it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 78.30it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 53.92it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 74.73it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 50.56it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 72.91it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 80.24it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 77.62it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 67.

2025-10-15 17:33:59,687 - src.agents.recommendation_agent - INFO - 상위 25개 논문, 16개 데이터셋 순위 결정 완료
2025-10-15 17:33:59,687 - src.agents.recommendation_agent - INFO - LLM paper 추천 생성 시도 1/2
2025-10-15 17:33:59,688 - src.agents.recommendation_agent - INFO - ================================================================================
2025-10-15 17:33:59,689 - src.agents.recommendation_agent - INFO - LLM에게 전송하는 paper 프롬프트:
2025-10-15 17:33:59,689 - src.agents.recommendation_agent - INFO - # Task: Re-rank and Select Top 5 paper Recommendations
You are a research recommendation expert. Re-rank the candidates and select the top 5 most relevant items.
## Source Dataset:
Title: Police recorded crime trends in Victoria during the COVID-19 pandemic
Description: The Crime Statistics Agency (CSA) has released a research paper entitled Police-recorded crime trends in Victoria during the COVID-19 pandemic. The paper examines trends in criminal offences and family violence incidents to 30 June 2020, 

평가 진행: 100%|██████████| 17/17 [09:49<00:00, 34.69s/it]

   ✅ nDCG@3=0.0000, MRR@3=0.0000, Recall@3=0.0000 | nDCG@5=0.0000, MRR@5=0.0000, Recall@5=0.0000

✅ 평가 완료: 17/17개 케이스 성공


## 5. 평가 결과 분석 및 저장

### 5.1 평가 결과 저장 디렉토리 설정

In [17]:
# 타임스탬프 기반 결과 디렉토리 생성
timestamp = datetime.now().strftime('%Y%m%d%H%M')
eval_output_dir = os.path.join(project_root, 'figures', 'evaluation_results', '202510151653')
os.makedirs(eval_output_dir, exist_ok=True)

print(f"📂 평가 결과 저장 경로: {eval_output_dir}")

# 평가 설정 저장
config_to_save = {
    'model_name': settings.MODEL_NAME,
    'embedding_model': settings.EMBEDDING_MODEL,
    'max_tokens': settings.MAX_TOKENS,
    'temperature': settings.TEMPERATURE,
    'paper_hybrid_alpha': settings.PAPER_HYBRID_ALPHA,
    'paper_hybrid_beta': settings.PAPER_HYBRID_BETA,
    'dataset_hybrid_alpha': settings.DATASET_HYBRID_ALPHA,
    'dataset_hybrid_beta': settings.DATASET_HYBRID_BETA,
    'dev_mode': settings.DEV_MODE,
    'evaluation_timestamp': timestamp,
    'testset_path': testset_path,
    'num_test_cases': len(all_test_cases)
}

config_path = os.path.join(eval_output_dir, 'config.json')
with open(config_path, 'w', encoding='utf-8') as f:
    json.dump(config_to_save, f, indent=2, ensure_ascii=False)

print(f"✅ 설정 저장 완료")
print(f"\n📋 설정 내용:")
print(json.dumps(config_to_save, indent=2, ensure_ascii=False))

📂 평가 결과 저장 경로: /home/infidea/backup-data/paper-reco-agent/figures/evaluation_results/202510151653
✅ 설정 저장 완료

📋 설정 내용:
{
  "model_name": "google/gemma-2-9b-it",
  "embedding_model": "intfloat/multilingual-e5-large",
  "max_tokens": 512,
  "temperature": 0.1,
  "paper_hybrid_alpha": 0.8,
  "paper_hybrid_beta": 0.2,
  "dataset_hybrid_alpha": 0.6,
  "dataset_hybrid_beta": 0.4,
  "dev_mode": false,
  "evaluation_timestamp": "202510151743",
  "testset_path": "/home/infidea/backup-data/paper-reco-agent/data/test/testset_filtered.json",
  "num_test_cases": 17
}


### 5.2 전체 메트릭 계산

In [18]:
if not all_eval_results:
    print("⚠️  평가 결과가 없습니다")
else:
    # DataFrame 변환
    df_results = pd.DataFrame(all_eval_results)

    print("\n" + "=" * 80)
    print("📊 평균 평가 메트릭")
    print("=" * 80)

    # 메트릭 요약 계산
    metrics_summary = {}

    for k in k_values:
        print(f"\n📏 k={k} 결과:")
        print("-" * 80)

        k_metrics = {}

        for metric_type in ['overall', 'paper', 'dataset']:
            type_label = {'overall': '전체', 'paper': '논문', 'dataset': '데이터셋'}[metric_type]
            print(f"\n  {type_label}:")

            type_metrics = {}
            for metric in ['ndcg', 'mrr', 'recall', 'precision']:
                col_name = f'{metric_type}_{metric}@{k}'
                if col_name in df_results.columns:
                    mean_val = df_results[col_name].mean()
                    std_val = df_results[col_name].std()
                    type_metrics[metric] = {'mean': mean_val, 'std': std_val}
                    print(f"    • {metric.upper():10s}: {mean_val:.4f} ± {std_val:.4f}")

            k_metrics[metric_type] = type_metrics

        metrics_summary[f'k{k}'] = k_metrics

    # 통계
    print("\n" + "=" * 80)
    print("📈 기본 통계")
    print("=" * 80)
    print(f"  평균 추천 논문 수: {df_results['num_predicted_papers'].mean():.2f}개")
    print(f"  평균 추천 데이터셋 수: {df_results['num_predicted_datasets'].mean():.2f}개")
    print(f"  평균 관련 논문 수: {df_results['num_relevant_papers'].mean():.2f}개")
    print(f"  평균 관련 데이터셋 수: {df_results['num_relevant_datasets'].mean():.2f}개")
    print(f"  평균 처리 시간: {df_results['processing_time_ms'].mean():.0f} ms")


📊 평균 평가 메트릭

📏 k=3 결과:
--------------------------------------------------------------------------------

  전체:
    • NDCG      : 0.1176 ± 0.3321
    • MRR       : 0.1176 ± 0.3321
    • RECALL    : 0.0135 ± 0.0423
    • PRECISION : 0.0392 ± 0.1107

  논문:
    • NDCG      : 0.1176 ± 0.3321
    • MRR       : 0.1176 ± 0.3321
    • RECALL    : 0.0191 ± 0.0556
    • PRECISION : 0.0392 ± 0.1107

  데이터셋:
    • NDCG      : 0.3482 ± 0.4863
    • MRR       : 0.3529 ± 0.4926
    • RECALL    : 0.1049 ± 0.1814
    • PRECISION : 0.1373 ± 0.2061

📏 k=5 결과:
--------------------------------------------------------------------------------

  전체:
    • NDCG      : 0.1332 ± 0.3180
    • MRR       : 0.1294 ± 0.3312
    • RECALL    : 0.0214 ± 0.0505
    • PRECISION : 0.0471 ± 0.1125

  논문:
    • NDCG      : 0.1104 ± 0.3125
    • MRR       : 0.1176 ± 0.3321
    • RECALL    : 0.0265 ± 0.0752
    • PRECISION : 0.0382 ± 0.1111

  데이터셋:
    • NDCG      : 0.3393 ± 0.4741
    • MRR       : 0.3529 ± 0.4926
    • REC

### 5.3 카테고리별 분석

In [19]:
if all_eval_results:
    print("\n" + "=" * 80)
    print("📊 카테고리별 메트릭 (k=5)")
    print("=" * 80)

    category_metrics = {}

    for category in sorted(df_results['category'].unique()):
        category_data = df_results[df_results['category'] == category]

        print(f"\n📁 {category} (n={len(category_data)}):")
        print("-" * 80)

        cat_metrics = {}
        for metric_type in ['overall', 'paper', 'dataset']:
            type_label = {'overall': '전체', 'paper': '논문', 'dataset': '데이터셋'}[metric_type]
            print(f"  {type_label}:")

            type_metrics = {}
            for metric in ['ndcg', 'mrr', 'recall', 'precision']:
                col_name = f'{metric_type}_{metric}@5'
                if col_name in category_data.columns:
                    mean_val = category_data[col_name].mean()
                    type_metrics[metric] = mean_val
                    print(f"    • {metric.upper():10s}: {mean_val:.4f}")

            cat_metrics[metric_type] = type_metrics

        category_metrics[category] = cat_metrics


📊 카테고리별 메트릭 (k=5)

📁 국내 (n=15):
--------------------------------------------------------------------------------
  전체:
    • NDCG      : 0.1509
    • MRR       : 0.1467
    • RECALL    : 0.0242
    • PRECISION : 0.0533
  논문:
    • NDCG      : 0.1251
    • MRR       : 0.1333
    • RECALL    : 0.0300
    • PRECISION : 0.0433
  데이터셋:
    • NDCG      : 0.3845
    • MRR       : 0.4000
    • RECALL    : 0.1467
    • PRECISION : 0.1200

📁 해외 (n=2):
--------------------------------------------------------------------------------
  전체:
    • NDCG      : 0.0000
    • MRR       : 0.0000
    • RECALL    : 0.0000
    • PRECISION : 0.0000
  논문:
    • NDCG      : 0.0000
    • MRR       : 0.0000
    • RECALL    : 0.0000
    • PRECISION : 0.0000
  데이터셋:
    • NDCG      : 0.0000
    • MRR       : 0.0000
    • RECALL    : 0.0000
    • PRECISION : 0.0000


### 5.4 최종 결과 저장

**📁 저장되는 파일:**
1. **`EVALUATION_SUMMARY.txt`** ⭐ - 가장 먼저 봐야 할 요약 리포트
2. **`detailed_results.csv`** - 엑셀에서 열어볼 수 있는 상세 데이터
3. **`config.json`** - 실험 재현을 위한 설정 정보
4. **`metrics.json`** - 프로그램으로 읽을 수 있는 메트릭 데이터

In [20]:
if all_eval_results:
    print("\n💾 결과 저장 중...\n")

    # 1. 요약 리포트 생성 (가장 중요!)
    report_lines = []
    report_lines.append("=" * 80)
    report_lines.append("평가 요약 리포트")
    report_lines.append("=" * 80)
    report_lines.append("")
    report_lines.append(f"📅 평가 시각: {timestamp}")
    report_lines.append(f"🤖 LLM 모델: {settings.MODEL_NAME}")
    report_lines.append(f"🔍 임베딩 모델: {settings.EMBEDDING_MODEL}")
    report_lines.append(f"📊 테스트 케이스: {len(all_eval_results)}/{len(all_test_cases)} 성공")
    report_lines.append("")
    report_lines.append("=" * 80)
    report_lines.append("주요 성능 지표 (k=5)")
    report_lines.append("=" * 80)
    report_lines.append("")

    # k=5 전체 메트릭
    for metric in ['ndcg', 'mrr', 'recall', 'precision']:
        col_name = f'overall_{metric}@5'
        if col_name in df_results.columns:
            mean_val = df_results[col_name].mean()
            report_lines.append(f"  {metric.upper():10s} @5: {mean_val:.4f}")

    report_lines.append("")
    report_lines.append("=" * 80)
    report_lines.append("타입별 성능 (k=5)")
    report_lines.append("=" * 80)

    for metric_type in ['paper', 'dataset']:
        type_label = {'paper': '논문', 'dataset': '데이터셋'}[metric_type]
        report_lines.append("")
        report_lines.append(f"[{type_label}]")
        for metric in ['ndcg', 'mrr', 'recall', 'precision']:
            col_name = f'{metric_type}_{metric}@5'
            if col_name in df_results.columns:
                mean_val = df_results[col_name].mean()
                report_lines.append(f"  {metric.upper():10s}: {mean_val:.4f}")

    report_lines.append("")
    report_lines.append("=" * 80)
    report_lines.append("카테고리별 nDCG@5")
    report_lines.append("=" * 80)
    report_lines.append("")

    for category in sorted(df_results['category'].unique()):
        category_data = df_results[df_results['category'] == category]
        mean_val = category_data['overall_ndcg@5'].mean()
        report_lines.append(f"  {category:30s}: {mean_val:.4f} (n={len(category_data)})")

    report_lines.append("")
    report_lines.append("=" * 80)
    report_lines.append("전체 메트릭 (모든 k 값)")
    report_lines.append("=" * 80)

    for k in k_values:
        report_lines.append("")
        report_lines.append(f"k={k}:")
        report_lines.append("-" * 40)
        for metric_type in ['overall', 'paper', 'dataset']:
            type_label = {'overall': '전체', 'paper': '논문', 'dataset': '데이터셋'}[metric_type]
            report_lines.append(f"  [{type_label}]")
            for metric in ['ndcg', 'mrr', 'recall', 'precision']:
                col_name = f'{metric_type}_{metric}@{k}'
                if col_name in df_results.columns:
                    mean_val = df_results[col_name].mean()
                    std_val = df_results[col_name].std()
                    report_lines.append(f"    {metric.upper():10s}: {mean_val:.4f} ± {std_val:.4f}")

    report_lines.append("")
    report_lines.append("=" * 80)

    # 요약 리포트 저장
    summary_path = os.path.join(eval_output_dir, 'EVALUATION_SUMMARY.txt')
    with open(summary_path, 'w', encoding='utf-8') as f:
        f.write('\n'.join(report_lines))
    print(f"✅ 1. 요약 리포트 저장: EVALUATION_SUMMARY.txt")

    # 2. 상세 CSV 저장
    csv_path = os.path.join(eval_output_dir, 'detailed_results.csv')
    df_results.to_csv(csv_path, index=False, encoding='utf-8-sig')
    print(f"✅ 2. 상세 결과 CSV 저장: detailed_results.csv")

    # 3. 메트릭 JSON 저장
    all_metrics = {
        'summary': metrics_summary,
        'by_category': category_metrics,
        'individual_results': all_eval_results
    }

    metrics_path = os.path.join(eval_output_dir, 'metrics.json')
    with open(metrics_path, 'w', encoding='utf-8') as f:
        json.dump(all_metrics, f, indent=2, ensure_ascii=False)
    print(f"✅ 3. 메트릭 JSON 저장: metrics.json")

    print(f"\n" + "=" * 80)
    print(f"📂 모든 결과가 저장되었습니다:")
    print(f"   {eval_output_dir}")
    print(f"" + "=" * 80)
    print(f"\n⭐ 결과 확인: EVALUATION_SUMMARY.txt 파일을 먼저 읽어보세요!")

    # 요약 리포트 미리보기
    print(f"\n" + "=" * 80)
    print("📄 요약 리포트 미리보기:")
    print("=" * 80)
    print('\n'.join(report_lines[:50]))  # 처음 50줄만 표시
    if len(report_lines) > 50:
        print("\n... (전체 내용은 EVALUATION_SUMMARY.txt 참고) ...")
else:
    print("⚠️  저장할 결과가 없습니다")


💾 결과 저장 중...

✅ 1. 요약 리포트 저장: EVALUATION_SUMMARY.txt
✅ 2. 상세 결과 CSV 저장: detailed_results.csv
✅ 3. 메트릭 JSON 저장: metrics.json

📂 모든 결과가 저장되었습니다:
   /home/infidea/backup-data/paper-reco-agent/figures/evaluation_results/202510151653

⭐ 결과 확인: EVALUATION_SUMMARY.txt 파일을 먼저 읽어보세요!

📄 요약 리포트 미리보기:
평가 요약 리포트

📅 평가 시각: 202510151743
🤖 LLM 모델: google/gemma-2-9b-it
🔍 임베딩 모델: intfloat/multilingual-e5-large
📊 테스트 케이스: 17/17 성공

주요 성능 지표 (k=5)

  NDCG       @5: 0.1332
  MRR        @5: 0.1294
  RECALL     @5: 0.0214
  PRECISION  @5: 0.0471

타입별 성능 (k=5)

[논문]
  NDCG      : 0.1104
  MRR       : 0.1176
  RECALL    : 0.0265
  PRECISION : 0.0382

[데이터셋]
  NDCG      : 0.3393
  MRR       : 0.3529
  RECALL    : 0.1294
  PRECISION : 0.1059

카테고리별 nDCG@5

  국내                            : 0.1509 (n=15)
  해외                            : 0.0000 (n=2)

전체 메트릭 (모든 k 값)

k=3:
----------------------------------------
  [전체]
    NDCG      : 0.1176 ± 0.3321
    MRR       : 0.1176 ± 0.3321

... (전체 내용은 EVALUATION_SUMM

## 7. 평가 완료

**결과 파일 안내**
- 평가 결과는 `figures/evaluation_results/{timestamp}/` 디렉토리에 저장됩니다.

**파일 읽는 순서:**
1. `EVALUATION_SUMMARY.txt` ⭐⭐⭐
   - 가장 먼저 읽어야 할 파일
   - 주요 성능 지표와 카테고리별 결과 요약
   - 텍스트 에디터로 바로 열어볼 수 있음

2. `detailed_results.csv`
   - 엑셀이나 스프레드시트로 열어서 상세 분석
   - 각 테스트 케이스별 메트릭 포함
   - 필터링, 정렬, 차트 생성 가능

3. `config.json`
   - 실험 재현을 위한 모든 설정 정보
   - 모델 설정, 하이브리드 가중치 등

4. `metrics.json`
   - 프로그램으로 읽을 수 있는 메트릭 데이터
   - 자동화된 분석이나 비교에 사용

**주요 메트릭 설명**
- nDCG@k: 순위를 고려한 추천 품질 (0~1, 높을수록 좋음)
- MRR@k: 첫 번째 관련 항목의 순위 (0~1, 높을수록 좋음)
- Recall@k: 관련 항목 중 추천된 비율 (0~1, 높을수록 좋음)
- Precision@k: 추천 항목 중 관련된 비율 (0~1, 높을수록 좋음)